In [ ]:
import geopandas as gpd
from shapely.geometry import Point

nyc_taxi_zones = gpd.read_file(filename='taxi_zones/taxi_zones.shp')
nyc_taxi_zones.to_crs(epsg=4326,inplace=True)
nyc_taxi_zones['point'] = nyc_taxi_zones.representative_point()
nyc_taxi_zones.head()

In [ ]:
import re

def get_jwd(point):
    fs = re.findall('\((.*)\)',str(point))
    if fs:
        lon,lat = fs[0].strip(' ').split(' ')
        lon = float(lon)
        lat = float(lat)
        return [lon,lat]


nyc_taxi_zones['lon'] = nyc_taxi_zones['point'].apply(lambda x:get_jwd(x)[0])
nyc_taxi_zones['lat'] = nyc_taxi_zones['point'].apply(lambda x:get_jwd(x)[1])

nyc_taxi_zones.head()

In [ ]:
long_min = -74.10
long_max = -73.70
lat_min = 40.58
lat_max = 40.90

global mark

mark = 0


def get_locationId(longitude,latitude):
    global mark
    mark += 1
    if mark % 10000 == 0:
        print(mark/100000)
    if longitude >= long_min and longitude <= long_max and latitude >= lat_min and latitude <= lat_max:
        pnts = Point(longitude,latitude)
        for i,j in zip(nyc_taxi_zones['LocationID'].values,nyc_taxi_zones['geometry'].values):
            if pnts.within(j):
                return i
    return -1

In [ ]:
import pandas as pd


df201507 = pd.read_csv('yellow_tripdata_2015-07.csv')
df201507.head()

In [ ]:
import time
t1 = time.time()
df201507['PULocationID'] = df201507[['pickup_longitude','pickup_latitude']].apply(lambda row: get_locationId(row['pickup_longitude'], row['pickup_latitude']), axis=1)
df201507['DOLocationID'] = df201507[['dropoff_longitude','dropoff_latitude']].apply(lambda row: get_locationId(row['dropoff_longitude'], row['dropoff_latitude']), axis=1)
t2 = time.time()
print(str(int(t2-t1)) + 'sec')
df201507.head()
df201507.to_csv('yellow_tripdata_2015-07_add.csv')

In [ ]:
nyc_taxi_zones = nyc_taxi_zones[['LocationID','zone','borough','lon','lat']]

In [ ]:
df201507_pickup = pd.merge(left=df201507,right=nyc_taxi_zones,how='inner',left_on='PULocationID',right_on='LocationID')
df201507_pickup['pickup_longitude'] = df201507_pickup['lon']
df201507_pickup['pickup_latitude'] = df201507_pickup['lat']
df201507_pickup['pickup_zone'] = df201507_pickup['zone']
df201507_pickup['pickup_borough'] = df201507_pickup['borough']
df201507_pickup.head()

In [ ]:
pick_up = list(df201507_pickup.columns)
pick_up.remove('LocationID')
pick_up.remove('zone')
pick_up.remove('borough')
pick_up.remove('lon')
pick_up.remove('lat')
print(pick_up)

In [ ]:
df201507 = df201507_pickup[pick_up]

In [ ]:
df201507_dropoff = pd.merge(left=df201507,right=nyc_taxi_zones,how='inner',left_on='DOLocationID',right_on='LocationID')
df201507_dropoff['dropoff_longitude'] = df201507_dropoff['lon']
df201507_dropoff['dropoff_latitude'] = df201507_dropoff['lat']
df201507_dropoff['dropoff_zone'] = df201507_dropoff['zone']
df201507_dropoff['dropoff_borough'] = df201507_dropoff['borough']
df201507_dropoff.head()

In [ ]:
dropoff = list(df201507_dropoff.columns)
dropoff.remove('LocationID')
dropoff.remove('zone')
dropoff.remove('borough')
dropoff.remove('lon')
dropoff.remove('lat')
print(dropoff)

In [ ]:
df201507 = df201507_dropoff[dropoff]
df201507.to_csv('yellow_tripdata_2015-07_clean.csv')

In [ ]:
df201807 = pd.read_csv('yellow_tripdata_2018-07.csv')
df201807.head()

In [ ]:
df201807_pickup = pd.merge(left=df201807,right=nyc_taxi_zones,how='inner',left_on='PULocationID',right_on='LocationID')
df201807_pickup['pickup_longitude'] = df201807_pickup['lon']
df201807_pickup['pickup_latitude'] = df201807_pickup['lat']
df201807_pickup['pickup_zone'] = df201807_pickup['zone']
df201807_pickup['pickup_borough'] = df201807_pickup['borough']
df201807_pickup.head()

In [ ]:
pick_up = list(df201807_pickup.columns)
pick_up.remove('LocationID')
pick_up.remove('zone')
pick_up.remove('borough')
pick_up.remove('lon')
pick_up.remove('lat')
print(pick_up)

In [ ]:
df201807 = df201807_pickup[pick_up]

In [ ]:
df201807_dropoff = pd.merge(left=df201807,right=nyc_taxi_zones,how='inner',left_on='DOLocationID',right_on='LocationID')
df201807_dropoff['dropoff_longitude'] = df201807_dropoff['lon']
df201807_dropoff['dropoff_latitude'] = df201807_dropoff['lat']
df201807_dropoff['dropoff_zone'] = df201807_dropoff['zone']
df201807_dropoff['dropoff_borough'] = df201807_dropoff['borough']
df201807_dropoff.head()

In [ ]:
df201807 = df201807_dropoff[dropoff]
df201807.head()

In [ ]:
df201807.to_csv('yellow_tripdata_2018-07_clean.csv')